In [1]:
try:
    import transformers
except ImportError:
    !pip install transformers
    import transformers
try:
    import emoji
except ImportError:
    !pip install emoji
    import emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.8 MB/s 
     |████████████████████████████████| 596 kB 56.9 MB/s 
     |████████████████████████████████| 101 kB 6.1 MB/s 
     |████████████████████████████████| 6.6 MB 30.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 7.9 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=9c45932e53f2710e82201f12fc8504e0ab59bad7b6009a6a15c04e5f2c5384f5
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [2]:
local = False
local_run = False

In [31]:
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer, TFDistilBertForSequenceClassification, TFRobertaForSequenceClassification
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,f1_score

import random
import string
import re
import ast

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import tensorflow_datasets as tfds

if local_run:
    nltk.data.path.append('/Users/algin/VOLD/nltk_data')
    nltk.download('stopwords',download_dir='/Users/algin/VOLD/nltk_data')
    nltk.download('punkt',download_dir='/Users/algin/VOLD/nltk_data')
else:
    nltk.download('stopwords')
    nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
if local:
    path = '/Users/algin/Greenwich/MSc Project/models/'
else:
    from google.colab import drive
    drive.mount('/content/drive')
    path = 'drive/MyDrive/MSc Data Science/MSc Project/models/'

Mounted at /content/drive


In [41]:
def get_emoji_regexp():
    # Sort emoji by length to make sure multi-character emojis are
    # matched first
    emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
    pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
    return re.compile(pattern)

In [42]:
banned_list= string.punctuation
punctuation_reg_exp = "[" + banned_list + "]"
emoji_reg_exp = get_emoji_regexp()

def stemmer(text):
    tokenized = nltk.word_tokenize(text)
    ps = PorterStemmer()
    return ' '.join([ps.stem(words) for words in tokenized])

def clean_text(text,stem=True):
    text = text.replace('\r', '').replace('\n', ' ').lower()
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)

    text = [word for word in text.split() if word not in stop_words]
    text = ' '.join(text)

    text = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', text))
    
    text = re.sub(punctuation_reg_exp,"",text)

    text = re.sub("\s\s+" , " ", text)

    text = re.sub(emoji_reg_exp, r"", text)

    if stem:
        text = stemmer(text)
    return text

**Load Dataset and transformations**

In [43]:
full_train_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/mixed/mixed_train_data.csv?raw=true")
test_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/mixed/mixed_test_data.csv?raw=true")

In [44]:
train_data, val_data = train_test_split(full_train_data,test_size=5000,random_state=21)

In [45]:
def split_label_and_feature(data):
    return data['tweet_text'], data['is_cyberbullying']

In [46]:
X_train, y_train = split_label_and_feature(train_data)
X_val, y_val = split_label_and_feature(val_data)
X_test, y_test = split_label_and_feature(test_data)

**Prediction on Best XGB Model**

In [ ]:
file_name = path + "mixed_best_xgb.pkl"
xgb_model = pickle.load(open(file_name, "rb"))

In [ ]:
xgb_model.get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.6,
 'gamma': 0.5,
 'learning_rate': 0.06,
 'max_delta_step': 0,
 'max_depth': 10,
 'min_child_weight': 1,
 'missing': nan,
 'n_estimators': 800,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 0.8,
 'verbosity': 1}

In [52]:
def split_label_and_feature_xgb(data):
    return data['cleaned'], data['is_cyberbullying']

In [53]:
X_train_xgb, y_train = split_label_and_feature_xgb(train_data)
X_val_xgb, y_val = split_label_and_feature_xgb(val_data)
X_test_xgb, y_test = split_label_and_feature_xgb(test_data)

In [54]:
clf = CountVectorizer()
X_train_cv =  clf.fit_transform(X_train_xgb)

tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_cv)
X_train_tf = tf_transformer.transform(X_train_cv)

In [55]:
def xgb_prediction(text_list,preprocess=True):
    df = pd.Series(text_list)
    if preprocess:
        df = df.apply(clean_text)
    test_cv = clf.transform(df)
    test_tf = tf_transformer.transform(test_cv)
    predictions = xgb_model.predict(test_tf)
    return predictions

In [56]:
xgb_prediction(["I believe in Christianity"])

array([0])

In [57]:
xgb_prediction(["What a good day"])

array([0])

In [58]:
xgb_prediction(["Muslims are terrorists"])

array([1])

In [59]:
xgb_prediction(["You are an asshole"])

array([1])

In [60]:
xgb_test_predictions = xgb_prediction(X_test_xgb,preprocess=False)

In [61]:
confusion_matrix(y_test,xgb_test_predictions)

array([[4703,  297],
       [ 801, 4199]])

In [62]:
f1_score(y_test,xgb_test_predictions)

0.8843723673125526

**Prediction on Transformer Model**

In [65]:
distilbert_model = TFDistilBertForSequenceClassification.from_pretrained(path + "mixed-distilbert--without-stem-94")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Some layers from the model checkpoint at drive/MyDrive/MSc Data Science/MSc Project/models/mixed-distilbert--without-stem-94 were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at drive/MyDrive/MSc Data Science/MSc Project/models/mixed-distilbert--without-stem-94 and are newly initialized: ['dropout_95']
You should probably TRAIN this model on a down-stre

In [66]:
def transformer_prediction(text_list,preprocess=True,return_one=False):  
    #tokenize the text
    if preprocess:
        new_list = []
        for each_text in text_list:
            new_list.append(clean_text(each_text,stem=False))
    else:
        new_list = text_list
    encodings = tokenizer(new_list, 
                          truncation=True, 
                          padding=True)
    #transform to tf.Dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(encodings)))
    #predict
    preds = distilbert_model.predict(dataset.batch(1)).logits  
    
    #transform to array with probabilities
    res = tf.nn.softmax(preds, axis=1).numpy()

    if return_one:
        return res.argmax(axis=1)  
    
    return res

In [67]:
transformer_prediction(["What a good day"])

array([[0.97284615, 0.02715382]], dtype=float32)

In [ ]:
transformer_prediction(["Muslims are terrorists"])

array([[0.00566342, 0.99433655]], dtype=float32)

In [ ]:
transformer_prediction(["You are an asshole"])

array([[0.00983728, 0.99016273]], dtype=float32)

In [ ]:
transformer_prediction(["I believe in Christianity"])

array([[0.86140543, 0.13859463]], dtype=float32)

In [ ]:
check_X = X_test#.sample(100)
check_y = y_test[check_X.index]
check_X = check_X.tolist()

In [ ]:
distil_predictions = transformer_prediction(check_X,return_one=True,preprocess=False)

In [ ]:
confusion_matrix(check_y,distil_predictions)

array([[4511,  489],
       [ 663, 4337]])

In [ ]:
f1_score(check_y,distil_predictions)

0.8827600244249948

**Prediction on Roberta Model**

In [32]:
roberta_model = TFRobertaForSequenceClassification.from_pretrained(path + "mixed-roberta")
roberta_tokenizer = AutoTokenizer.from_pretrained("roberta-base")

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at drive/MyDrive/MSc Data Science/MSc Project/models/mixed-roberta.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [12]:
batch_size = 64

In [13]:
max_length = 128
def convert_example_to_feature(text):
  return roberta_tokenizer.encode_plus(text,
                                       add_special_tokens=True,
                                       max_length=max_length,
                                       pad_to_max_length=True,
                                       return_attention_mask=True,
  )

def map_example_to_dict(input_ids, attention_masks, label):
    return {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
           }, label

def encode_examples(ds, limit=-1):
     # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  attention_mask_list = []
  label_list = []
  if (limit > 0):
    ds = ds.take(limit)
  for text, label in tfds.as_numpy(ds):
    bert_input = convert_example_to_feature(text.decode())
    input_ids_list.append(bert_input['input_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    label_list.append([label])
  return tf.data.Dataset.from_tensor_slices((input_ids_list,
                                             attention_mask_list,
                              label_list)).map(map_example_to_dict)

In [27]:
def predict_proba(text_list, model, preprocess=True, return_one=True):
    if preprocess:
        new_list = []
        for each_text in text_list:
            new_list.append(clean_text(each_text,stem=False))
    else:
        new_list = text_list
    print(new_list)
    df = pd.DataFrame(new_list, columns=['text'])
    df['label'] = 0
    sentences_modified = tf.data.Dataset.from_tensor_slices((df['text'],df['label']))
    ds_encoded = encode_examples(sentences_modified).batch(batch_size)

    # preds_raw = tf.nn.softmax(model.predict(ds_encoded).logits)
    # preds = tf.math.argmax(preds_raw, axis=1)

    preds = model.predict(ds_encoded).logits  
    
    #transform to array with probabilities
    res = tf.nn.softmax(preds, axis=1).numpy()

    if return_one:
        return res.argmax(axis=1)  
    
    return res

In [33]:
check_X = X_test#.sample(100)
check_y = y_test[check_X.index]
check_X = check_X.tolist()

In [34]:
roberta_predictions = predict_proba(check_X,roberta_model,return_one=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[' accuse good faith editors enacting consensus vandalism', 'i add that getting together coffee always codeword getting used many times gta think spoofs spoofs gta minigame instead referring tired old nudgenudge winkwink coffee surfbruddah ', 'btw hard feelings delete afterwards', ' ani hello message sent inform currently discussion wikipediaadministrators noticeboardincidents regarding issue may involved thread concerns regarding brownhairedgirl thank you talk ', 'early life william hosking travelled australia parents later returned uk', 'names individual games putting names individual games seems bit silly unless theres list games featured cube article explaining are', 'i got married btw like glasses', '0032 wikipedia signpost 12 april 2010 read signpost full · singlepage · unsubscribe ·', 'sees mother jones article titled feminism needs men laughs closes tab smashes window', 'its also specific complaint hard know remove immediately complainant specified remove maybe news flash could

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [35]:
confusion_matrix(check_y,roberta_predictions)

array([[4495,  505],
       [ 489, 4511]])

In [36]:
f1_score(check_y,roberta_predictions)

0.900758785942492